In [2]:
import torch as th
import numpy as np

In [5]:
import pickle

# 定义新文件路径
goodreads_children_path = 'Dataset/goodreads_children/processed/children.pkl'

# 加载 goodreads_children.pkl 文件
with open(goodreads_children_path, 'rb') as f:
    goodreads_children_data = pickle.load(f)

# 打印加载的数据
print(goodreads_children_data)

Data(edge_index=[2, 734640], text_nodes=[216613], text_edges=[734640], text_node_labels=[216613], edge_score=[734640])


In [7]:
product_labels = goodreads_children_data.text_node_labels


In [9]:
from sklearn.preprocessing import MultiLabelBinarizer

In [10]:
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(product_labels)

TypeError: 'int' object is not iterable

In [64]:
dgl_graph = goodreads_children_data
test_ratio=0.1
val_ratio=0.2
random_seed=42
neg_len="1000"
path=None
way="random"

In [65]:
graph = dgl_graph

eids = np.arange(graph.num_edges)
eids = np.random.permutation(eids)

u, v = graph.edge_index

test_size = int(len(eids) * test_ratio)
val_size = int(len(eids) * val_ratio)

test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
val_pos_u, val_pos_v = (
	u[eids[test_size : test_size + val_size]],
	v[eids[test_size : test_size + val_size]],
)
train_pos_u, train_pos_v = (
	u[eids[test_size + val_size :]],
	v[eids[test_size + val_size :]],
)

valid_target_neg = th.randint(
	0, graph.num_nodes, [len(val_pos_u), int(neg_len)], dtype=th.long
)  # TODO: filter some edges in graph
test_target_neg = th.randint(
	0, graph.num_nodes, [len(test_pos_u), int(neg_len)], dtype=th.long
)  # TODO: filter some edges in graph



/home/lizhuofeng/.local/lib/python3.10/site-packages/torch_geometric/data/storage.py:452: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'text_edges', 'text_nodes'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [76]:
from torch_sparse import SparseTensor


adj_t = SparseTensor.from_edge_index(goodreads_children_data.edge_index, sparse_sizes=(goodreads_children_data.num_nodes, goodreads_children_data.num_nodes)).t()


In [73]:
goodreads_children_data.num_nodes

/home/lizhuofeng/.local/lib/python3.10/site-packages/torch_geometric/data/storage.py:452: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'text_edges', 'text_nodes'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


60785

In [77]:
adj_t.size(0), adj_t.size(1)

(60785, 60785)

In [2]:
from torch.utils.data import DataLoader

In [3]:
for perm in DataLoader(range(10), 2, shuffle=True):
    print(perm)

tensor([4, 9])
tensor([6, 8])
tensor([0, 5])
tensor([7, 3])
tensor([1, 2])
